In [ ]:
import MDAnalysis as mda
from MDAnalysis.analysis import rms, align
import plotly.graph_objects as go
import numpy as np
import pickle
from src.data.plot.plot_mdcath import hamming_distance
from Levenshtein import distance as levenshtein_distance

In [ ]:
u = mda.Universe("../tmp/output/trajectories/12asA00.pdb", 
                 "../tmp/output/trajectories/12asA00_320_0.xtc")

n_frames = len(u.trajectory)
dt = u.trajectory.dt
time = np.arange(n_frames) * dt

rmsd_values = np.zeros(n_frames)

u.trajectory[0]
first_frame_coords = u.select_atoms('backbone').positions.copy()

ref = u.copy()
ref.trajectory[0]

aligner = align.AlignTraj(u, ref, select='backbone', in_memory=True)
aligner.run()

for i, ts in enumerate(u.trajectory):
    current_pos = u.select_atoms('backbone').positions
    rmsd = rms.rmsd(current_pos, first_frame_coords)
    rmsd_values[i] = rmsd

fig = go.Figure()
fig.add_trace(go.Scatter(x=time, y=rmsd_values,
                        mode='lines',
                        name='RMSD'))

fig.update_layout(
    title='Backbone RMSD vs First Frame',
    xaxis_title='Time (ps)',
    yaxis_title='RMSD (Å)',
    showlegend=True,
    template='plotly_white'
)

fig.show()

In [ ]:


u = mda.Universe("../tmp/output/trajectories/12asA00.pdb", 
                 "../tmp/output/trajectories/12asA00_320_0.xtc")

n_frames = len(u.trajectory)
dt = u.trajectory.dt
time = np.arange(n_frames) * dt

rmsd_values = np.zeros(n_frames - 1)

ref = u.copy()
ref.trajectory[0]

aligner = align.AlignTraj(u, ref, select='backbone', in_memory=True)
aligner.run()

prev_pos = None
for i, ts in enumerate(u.trajectory):
    if i > 0:
        current_pos = u.select_atoms('backbone').positions
        rmsd = rms.rmsd(current_pos, prev_pos)
        rmsd_values[i-1] = rmsd
    prev_pos = u.select_atoms('backbone').positions.copy()

fig = go.Figure()
fig.add_trace(go.Scatter(x=time[1:], y=rmsd_values,
                        mode='lines',
                        name='RMSD'))

fig.update_layout(
    title='Backbone RMSD Between Consecutive Frames',
    xaxis_title='Time (ps)',
    yaxis_title='RMSD (Å)',
    showlegend=True,
    template='plotly_white'
)

fig.show()

In [4]:
with open('../tmp/data/tokenized/mdcath/mdcath_foldseek_trajectories.pkl', 'rb') as f:
    tokenized_mdcath = pickle.load(f)

In [ ]:
print(*tokenized_mdcath['12asA00_320_0'][164:167], sep='\n')

In [ ]:
print('hi')

In [ ]:
distances = []
dist_name = "Levenshtein"
for i in range(0, len(tokenized_mdcath['12asA00_320_0'])):
    distance = levenshtein_distance(tokenized_mdcath['12asA00_320_0'][0], tokenized_mdcath['12asA00_320_0'][i])
    distances.append(distance)

fig = go.Figure()
fig.add_trace(go.Scatter(x=time[1:], y=distances,
                        mode='lines',
                        name=dist_name))

fig.update_layout(
    title=f'{dist_name} Distance vs First Frame',
    xaxis_title='Time (ps)',
    yaxis_title=f'{dist_name} Distance',
    showlegend=True,
    template='plotly_white'
)

fig.show()

In [ ]:
# Calculate hamming distances between consecutive frames
distances = []
dist_name = "Levenshtein"
for i in range(1, len(tokenized_mdcath['12asA00_320_0'])):
    distance = levenshtein_distance(tokenized_mdcath['12asA00_320_0'][i-1], tokenized_mdcath['12asA00_320_0'][i])
    distances.append(distance)

fig = go.Figure()
fig.add_trace(go.Scatter(x=time[1:], y=distances,
                        mode='lines',
                        name='Distance'))

fig.update_layout(
    title=f'{dist_name} Distance Between Consecutive Frames',
    xaxis_title='Time ', 
    yaxis_title=f'{dist_name} Distance',
    showlegend=True,
    template='plotly_white'
)

fig.show()
